# 5次式の2次式と3次式の積への因数分解の仕方

* 黒木玄

In [1]:
using SymPy

# Override the Base.show definition of SymPy.jl:
# https://github.com/JuliaPy/SymPy.jl/blob/29c5bfd1d10ac53014fa7fef468bc8deccadc2fc/src/types.jl#L87-L105

@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::SymbolicObject)
    print(io, as_markdown("\\displaystyle " * sympy.latex(x, mode="plain", fold_short_frac=false)))
end
@eval SymPy function Base.show(io::IO, ::MIME"text/latex", x::AbstractArray{Sym})
    function toeqnarray(x::Vector{Sym})
        a = join(["\\displaystyle " * sympy.latex(x[i]) for i in 1:length(x)], "\\\\")
        """\\left[ \\begin{array}{r}$a\\end{array} \\right]"""
    end
    function toeqnarray(x::AbstractArray{Sym,2})
        sz = size(x)
        a = join([join("\\displaystyle " .* map(sympy.latex, x[i,:]), "&") for i in 1:sz[1]], "\\\\")
        "\\left[ \\begin{array}{" * repeat("r",sz[2]) * "}" * a * "\\end{array}\\right]"
    end
    print(io, as_markdown(toeqnarray(x)))
end

## 一般論

In [2]:
@vars a b c d e p q r s t x

(a, b, c, d, e, p, q, r, s, t, x)

In [3]:
f = x^5 + p*x^4 + q*x^3 + r*x^2 + s*x + t

   4      3      2              5
p*x  + q*x  + r*x  + s*x + t + x 

In [4]:
g = x^2 + a*x + b

           2
a*x + b + x 

In [5]:
h = x^3 + c*x^2 + d*x + e

   2              3
c*x  + d*x + e + x 

In [6]:
ghmf = sympy.poly(g*h - f, x)

Poly((a + c - p)*x**4 + (a*c + b + d - q)*x**3 + (a*d + b*c + e - r)*x**2 + (a
*e + b*d - s)*x + b*e - t, x, domain='ZZ[a,b,c,d,e,p,q,r,s,t]')

$gh=f$ となることと次のセルの内容がすべて $0$ になることは同値.

In [7]:
eq = ghmf.coeffs()

5-element Vector{Sym}:
         a + c - p
   a*c + b + d - q
 a*d + b*c + e - r
     a*e + b*d - s
           b*e - t

この方程式を使って $c,d,e$ を $a,b,p,q,r$ で表そう.

In [8]:
C = p - a

-a + p

In [9]:
D = q - a*C - b |> expand

 2              
a  - a*p - b + q

In [10]:
E = r - a*D - b*C |> expand

   3    2                          
- a  + a *p + 2*a*b - a*q - b*p + r

元の方程式にこれらを代入して $c,d,e$ を削除すると以下のようになる.

In [11]:
eq1 = eq .|> (F -> -F(c=>C, d=>D, e=>E).expand())

5-element Vector{Sym}:
                                                             0
                                                             0
                                                             0
 a^4 - a^3*p - 3*a^2*b + a^2*q + 2*a*b*p - a*r + b^2 - b*q + s
           a^3*b - a^2*b*p - 2*a*b^2 + a*b*q + b^2*p - b*r + t

$b\ne 0$ の場合に($t\ne 0$ ならば $b\ne 0$ となる), 最下段の方程式は次のセルのように書き直される($t$ に $be$ を代入しておく).

In [12]:
eqlast = eq1[end](t=>b*e) / b |> expand

 3    2                              
a  - a *p - 2*a*b + a*q + b*p + e - r

## 例1

$g = x^2 + 3x - 2$, $h = (x+1)^3-2 = x^3+3x^2+3x-1$ の場合にどうなるか.

In [13]:
GG = x^2 + 3x - 2

 2          
x  + 3*x - 2

In [14]:
HH = (x+1)^3-2 |> expand

 3      2          
x  + 3*x  + 3*x - 1

In [15]:
FF = GG*HH

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

In [16]:
FF = expand(FF)

 5      4       3      2          
x  + 6*x  + 10*x  + 2*x  - 9*x + 2

既存の函数を使えば因数分解は易しい.

In [17]:
factor(FF)

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

係数を $P,Q,R,S,T$ に代入する.

In [18]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
  6
 10
  2
 -9
  2

最下段の方程式の形の確認.

In [19]:
Eqlast = eqlast(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3      2                             
a  - 6*a  - 2*a*b + 10*a + 6*b + e - 2

$be = 2$ となる整数の組 $(b, e)$ 全体について, 最下段の方程式が整数解を持つ場合を探す.

In [20]:
[Eqlast(b=>k, e=>T/k) |> factor for k in (1, 2, -1, -2)]

4-element Vector{Sym}:
   a^3 - 6*a^2 + 8*a + 6
  a^3 - 6*a^2 + 6*a + 11
 a^3 - 6*a^2 + 12*a - 10
 (a - 3)*(a^2 - 3*a + 5)

最後の $b=-2$, $e=-1$ の場合にのみ整数解 $a=3$ が存在する.

In [21]:
BB, AA = -2, 3

(-2, 3)

この場合が実際に解になっていることの確認.

In [22]:
C, D, E

(-a + p, a^2 - a*p - b + q, -a^3 + a^2*p + 2*a*b - a*q - b*p + r)

In [23]:
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(3, 3, -1)

In [24]:
eq[end-1]

a*e + b*d - s

In [25]:
eq[end-1](a=>AA, b=>BB, d=>DD, e=>EE, s=>S)

0

In [26]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2          \ / 3      2          \
\x  + 3*x - 2/*\x  + 3*x  + 3*x - 1/

In [27]:
sol - FF |> simplify

0

## 例2

$f = x^5 - x^4 - 1$ の因数分解.

In [28]:
FF = x^5 - x^4 - 1

 5    4    
x  - x  - 1

既存の函数を使えば易しい.

In [29]:
factor(FF)

/ 2        \ / 3        \
\x  - x + 1/*\x  - x - 1/

よくある解答例では天下り的に「$x^3$ を足して引く」という方法を使っている.

$$
\begin{aligned}
x^5 - x^4 - 1
&=
x^5 - x^4 + x^3 - x^3 - 1
\\ &=
(x^5 - x^4 + x^3) - (x^3 + 1)
\\ &=
x^3(x^2-x+1) -(x+1)(x^2-x+1)
\\ &=
(x^2-x+1)(x^3-x-1).
\end{aligned}
$$

以下の方法であればそういう天下り的な操作が無用になる.

係数を $P,Q,R,S,T$ に代入する.

In [30]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
 -1
  0
  0
  0
 -1

最下段の方程式の形の確認.

In [31]:
Eqlast = eqlast(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3    2                
a  + a  - 2*a*b - b + e

$be=-1$ となる整数の組 $(b,d)$ の各々について, 最下段の方程式が整数解を持つかどうかを確認する.

In [32]:
[(B = k; E = T/k; Eqlast(b=>B, e=>E) |> factor) for k in (1, -1)]

2-element Vector{Sym}:
 (a + 1)*(a^2 - 2)
 (a + 1)*(a^2 + 2)

$(b,d)=(1,-1), (-1,1)$ の両方の場合に整数解 $a=-1$ を持つことがわかった.

これらが下から2段目の方程式も満たしているかを確認する.

In [33]:
BB, AA = 1, -1
eq[end-1](a=>AA, b=>BB, d=>D(a=>AA, b=>BB, p=>P, q=>Q), e=>E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T), s=>S)

0

In [34]:
BB, AA = -1, -1
eq[end-1](a=>AA, b=>BB, d=>D(a=>2, b=>4, p=>P, q=>Q), e=>E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T), s=>S)

-3

$(a, b) = -1, 1$ の場合のみが解になっていることがわかった.

以下は検算である.

In [35]:
AA, BB = -1, 1
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(0, -1, -1)

In [36]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2        \ / 3        \
\x  - x + 1/*\x  - x - 1/

In [37]:
sol - FF |> simplify

0

## 例3

$f = x^5 + 16x + 32$ の因数分解.

In [38]:
FF = x^5 + 16x + 32

 5            
x  + 16*x + 32

既存の函数を使えば易しい.

In [39]:
factor(FF)

/ 2          \ / 3      2    \
\x  + 2*x + 4/*\x  - 2*x  + 8/

係数を $P,Q,R,S,T$ に代入する.

In [40]:
_, P, Q, R, S, T = [FF.coeff(x, k) for k in 5:-1:0]

6-element Vector{Sym}:
  1
  0
  0
  0
 16
 32

最下段の方程式.

In [41]:
Eqlast = eqlast(p=>P, q=>Q, r=>R, s=>S, t=>T)

 3            
a  - 2*a*b + e

In [42]:
[(B = k; E = T/k; (b=B, e=E, eqlast = Eqlast(b=>B, e=>E) |> factor)) for k in ((2 .^ (0:5))..., (-2 .^ (0:5))...)]

12-element Vector{NamedTuple{(:b, :e, :eqlast), Tuple{Int64, Sym, Sym}}}:
 (b = 1, e = 32, eqlast = a^3 - 2*a + 32)
 (b = 2, e = 16, eqlast = a^3 - 4*a + 16)
 (b = 4, e = 8, eqlast = (a - 2)*(a^2 + 2*a - 4))
 (b = 8, e = 4, eqlast = a^3 - 16*a + 4)
 (b = 16, e = 2, eqlast = a^3 - 32*a + 2)
 (b = 32, e = 1, eqlast = a^3 - 64*a + 1)
 (b = -1, e = -32, eqlast = a^3 + 2*a - 32)
 (b = -2, e = -16, eqlast = (a - 2)*(a^2 + 2*a + 8))
 (b = -4, e = -8, eqlast = a^3 + 8*a - 8)
 (b = -8, e = -4, eqlast = a^3 + 16*a - 4)
 (b = -16, e = -2, eqlast = a^3 + 32*a - 2)
 (b = -32, e = -1, eqlast = a^3 + 64*a - 1)

In [43]:
BB, AA = 4, 2
eq[end-1](a=>AA, b=>BB, d=>D(a=>AA, b=>BB, p=>P, q=>Q), e=>E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T), s=>S)

0

In [44]:
BB, AA = -2, -16
eq[end-1](a=>AA, b=>BB, d=>D(a=>AA, b=>BB, p=>P, q=>Q), e=>E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T), s=>S)

-67092

解 $a = 2$, $b=4$ が見付かった.

In [45]:
AA, BB = 2, 4
CC, DD, EE = C(a=>AA, b=>BB, p=>P), D(a=>AA, b=>BB, p=>P, q=>Q), E(a=>AA, b=>BB, p=>P, q=>Q, r=>R, t=>T)

(-2, 0, 8)

In [46]:
sol = (x^2 + AA*x + BB)*(x^3 + CC*x^2 + DD*x + EE)

/ 2          \ / 3      2    \
\x  + 2*x + 4/*\x  - 2*x  + 8/

In [47]:
sol - FF |> simplify

0